In [27]:
from plexosdb import PlexosSQLite
import polars as pl


In [28]:
DEFAULT_QUERY_COLUMNS_SCHEMA = {  # NOTE: Order matters
    # "membership_id": pl.Int64,
    "parent_class_id": pl.Int32,
    "parent_object_id": pl.Int32,
    "parent_class_name": pl.String,
    "child_class_id": pl.Int32,
    "child_class_name": pl.String,
    "category": pl.String,
    "object_id": pl.Int32,
    "name": pl.String,
    "property_name": pl.String,
    "property_unit": pl.String,
    "property_value": pl.Float32,
    "band": pl.Int32,
    "date_to": pl.String,
    "date_from": pl.String,
    "memo": pl.String,
    "scenario": pl.String,
    "data_file": pl.String,
    "variable": pl.String,
    "timeslice": pl.String,
    "action": pl.String,
    "tag_action_id": pl.Int32,
}
xml_file = '/Users/kamrantehranchi/Local_Documents/FPA_Sienna/Projects/NVE/NVE_7_19_24/PLEXOS_DB_Nevada.xml'

In [29]:

query ="""
WITH scenario_cte AS (
    SELECT
        obj.name,
        obj.object_id,
        tag.data_id,
        mem.collection_id
    FROM
        t_membership AS mem
    INNER JOIN t_tag AS tag ON
        tag.object_id = mem.child_object_id
    INNER JOIN t_object AS obj ON
        mem.child_object_id = obj.object_id
    WHERE
        mem.child_class_id = 78
        AND mem.collection_id IN (1, 698, 706, 700) -- Collections belong to scenarios
),
text_cte AS (
    SELECT
        obj.object_id,
        obj.name AS nested_object,
        prop.name AS nested_property_name,
        text.value AS text,
        d.data_id,
        d.value,
        tag.data_id AS tag_data_id,
        date_from.date AS date_from,
        date_to.date AS date_to,
        memo.value AS memo,
        scenario.name AS scenario,
        text.class_id AS text_class_id,
        text.action_id AS text_action_id,
        class_text.name AS class_text_name,
        action.action_symbol AS action_symbol,
        tag.action_id AS tag_action_id
    FROM
        t_membership AS mem
    LEFT JOIN t_data AS d ON
        mem.membership_id = d.membership_id
    LEFT JOIN t_property AS prop ON
        prop.property_id = d.property_id
    LEFT JOIN t_memo_data AS memo ON
        memo.data_id = d.data_id
    LEFT JOIN t_date_from AS date_from ON
        d.data_id = date_from.data_id
    LEFT JOIN t_date_to AS date_to ON
        d.data_id = date_to.data_id
    INNER JOIN t_text AS text ON
        text.data_id = d.data_id
    INNER JOIN t_object AS obj ON
        mem.child_object_id = obj.object_id
    INNER JOIN t_tag AS tag ON
        tag.object_id = obj.object_id
    LEFT JOIN t_class AS class_text ON
        text.class_id = class_text.class_id
    LEFT JOIN t_action AS action ON
        text.action_id = action.action_id
    LEFT JOIN scenario_cte AS scenario ON
        d.data_id = scenario.data_id
)
SELECT
    class_parent.class_id AS parent_class_id,
    mem.parent_object_id as parent_object_id,
    class_parent.name AS parent_class,
    class_child.class_id AS child_class_id,
    class_child.name AS child_class,
    cat.name AS category,
    child_obj.object_id as object_id,
    child_obj.name as object_name,
    prop.name AS property_name,
    unit.value AS property_unit,
    data.value as property_value,
    IFNULL(band.band_id, 1) as band,
    COALESCE(date_from.date, nested_object.date_from) as date_from,
    COALESCE(date_to.date, nested_object.date_to) as date_to,
    COALESCE(memo.value, nested_object.memo) as memo,
    COALESCE(scenario.name, nested_object.scenario) AS scenario,
    MAX(CASE WHEN class_text_name = 'Data File' THEN text END) AS data_file,
    MAX(CASE WHEN class_text_name = 'Variable' THEN text END) AS variable,
    MAX(CASE WHEN class_text_name = 'Timeslice' THEN text END) AS timeslice,
    nested_object.action_symbol AS action_symbol,
    nested_object.tag_action_id AS tag_action_id
FROM
    t_membership AS mem
LEFT JOIN t_class AS class_parent ON
    mem.parent_class_id = class_parent.class_id
LEFT JOIN t_class AS class_child ON
    mem.child_class_id = class_child.class_id
LEFT JOIN t_collection AS collection ON
    collection.collection_id = mem.collection_id
LEFT JOIN t_object AS child_obj ON
    child_obj.object_id = mem.child_object_id
LEFT JOIN t_object AS parent_obj ON
    parent_obj.object_id = mem.parent_object_id
LEFT JOIN t_data AS data ON
    data.membership_id = mem.membership_id
LEFT JOIN t_memo_data AS memo ON
    memo.data_id = data.data_id
LEFT JOIN t_date_from AS date_from ON
    data.data_id = date_from.data_id
LEFT JOIN t_date_to AS date_to ON
    data.data_id = date_to.data_id
LEFT JOIN t_property AS prop ON
    data.property_id = prop.property_id
LEFT JOIN t_unit AS unit ON
    unit.unit_id = prop.unit_id
LEFT JOIN t_band AS band ON
    data.data_id = band.data_id
LEFT JOIN text_cte AS nested_object ON
    data.data_id = nested_object.tag_data_id
LEFT JOIN scenario_cte AS scenario ON
    scenario.data_id = data.data_id
LEFT JOIN t_category AS cat ON
    child_obj.category_id = cat.category_id
GROUP BY
    class_parent.class_id,
    mem.parent_object_id,
    class_parent.name,
    class_child.class_id,
    class_child.name,
    cat.name,
    child_obj.object_id,
    child_obj.name,
    prop.name,
    unit.value,
    data.value,
    IFNULL(band.band_id, 1),
    COALESCE(date_from.date, nested_object.date_from),
    COALESCE(date_to.date, nested_object.date_to),
    COALESCE(memo.value, nested_object.memo),
    COALESCE(scenario.name, nested_object.scenario),
    nested_object.action_symbol;
"""


In [30]:
qr = PlexosSQLite(xml_file).query(query)
df = pl.from_records(qr,  schema=DEFAULT_QUERY_COLUMNS_SCHEMA)
print(df)

2024-08-05 12:19:45.719 | DEBUG    | plexosdb.sqlite:_create_table_schema:774 - Using /Users/kamrantehranchi/Local_Documents/plexosdb/src/plexosdb/schema.sql for creating plexos schema.


shape: (8_678, 21)
┌────────────┬────────────┬────────────┬───────────┬───┬──────────┬───────────┬────────┬───────────┐
│ parent_cla ┆ parent_obj ┆ parent_cla ┆ child_cla ┆ … ┆ variable ┆ timeslice ┆ action ┆ tag_actio │
│ ss_id      ┆ ect_id     ┆ ss_name    ┆ ss_id     ┆   ┆ ---      ┆ ---       ┆ ---    ┆ n_id      │
│ ---        ┆ ---        ┆ ---        ┆ ---       ┆   ┆ str      ┆ str       ┆ str    ┆ ---       │
│ i32        ┆ i32        ┆ str        ┆ i32       ┆   ┆          ┆           ┆        ┆ i32       │
╞════════════╪════════════╪════════════╪═══════════╪═══╪══════════╪═══════════╪════════╪═══════════╡
│ 1          ┆ 1          ┆ System     ┆ 2         ┆ … ┆ null     ┆ null      ┆ null   ┆ null      │
│ 1          ┆ 1          ┆ System     ┆ 2         ┆ … ┆ null     ┆ null      ┆ null   ┆ null      │
│ 1          ┆ 1          ┆ System     ┆ 2         ┆ … ┆ null     ┆ null      ┆ null   ┆ null      │
│ 1          ┆ 1          ┆ System     ┆ 2         ┆ … ┆ null     ┆ null

/var/folders/00/vqry7y9s78q3rvf_m_jmk_bc0000gn/T/ipykernel_56769/2291346775.py:2: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df = pl.from_records(qr,  schema=DEFAULT_QUERY_COLUMNS_SCHEMA)


In [120]:
text_cte = """
   SELECT
        obj.object_id,
        obj.name AS nested_object_name,
        prop.name AS nested_property_name,
        text.value AS text,
        text.data_id AS text_data_id,
        tag.data_id AS tag_data_id,
        date_from.date AS date_from,
        date_to.date AS date_to,
        memo.value AS memo,
        class_text.name AS class_text_name,
        action.action_symbol AS action_symbol
    FROM
        t_membership AS mem
    LEFT JOIN t_data AS d ON
        mem.membership_id = d.membership_id
    LEFT JOIN t_property AS prop ON
        prop.property_id = d.property_id
    LEFT JOIN t_memo_data AS memo ON
        memo.data_id = d.data_id
    LEFT JOIN t_date_from AS date_from ON
        d.data_id = date_from.data_id
    LEFT JOIN t_date_to AS date_to ON
        d.data_id = date_to.data_id
    INNER JOIN t_text AS text ON
        text.data_id = d.data_id
    INNER JOIN t_object AS obj ON
        mem.child_object_id = obj.object_id
    INNER JOIN t_tag AS tag ON
        tag.object_id = obj.object_id
    LEFT JOIN t_class AS class_text ON
        text.class_id = class_text.class_id
    LEFT JOIN t_action AS action ON
        text.action_id = action.action_id
"""

schema = { 
    'parent_object_id': pl.Int32,
    'parent_object_name': pl.String,
    'parent_property_name': pl.String,
    'text': pl.String,
    'text_data_id': pl.Int32,
    'tag_data_id': pl.Int32,
    'date_from': pl.String,
    'date_to': pl.String,
    'memo': pl.String,
    'class_text_name': pl.String,
    'action_symbol': pl.String,
}

In [121]:
qr = PlexosSQLite(xml_file).query(text_cte)
df = pl.from_records(qr, schema=schema)
df.write_csv('text_cte.csv')

2024-08-06 17:28:29.070 | DEBUG    | plexosdb.sqlite:_create_table_schema:774 - Using /Users/kamrantehranchi/Local_Documents/plexosdb/src/plexosdb/schema.sql for creating plexos schema.
/var/folders/00/vqry7y9s78q3rvf_m_jmk_bc0000gn/T/ipykernel_56769/590497418.py:2: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df = pl.from_records(qr, schema=schema)


In [112]:
tag_cte = """
    SELECT
        mem.collection_id,
        obj.object_id AS child_object_id,
        obj.name AS child_object_name,
        prop.name AS child_property_name,
        tag.data_id AS tag_data_id,
        tag.object_id AS tag_object_id,
        tag_object.name AS tag_object_name,
        tag_object.class_id AS tag_object_class_id,
        date_from.date AS date_from,
        date_to.date AS date_to,
        memo.value AS memo,
        action.action_symbol AS action_symbol,
        tag.action_id AS tag_action_id,
        text.value AS text,
        text.class_id AS text_class_id
    FROM
        t_membership AS mem
    LEFT JOIN t_data AS d ON
        mem.membership_id = d.membership_id                             -- brings in data associated with the membership
    LEFT JOIN t_property AS prop ON                                     -- property "Start Cost" example
        prop.property_id = d.property_id
    LEFT JOIN t_memo_data AS memo ON
        memo.data_id = d.data_id
    LEFT JOIN t_date_from AS date_from ON
        d.data_id = date_from.data_id
    LEFT JOIN t_date_to AS date_to ON
        d.data_id = date_to.data_id
    INNER JOIN t_tag AS tag ON                                  -- tag table can include variable reference, and action
        tag.data_id = d.data_id
    INNER JOIN t_object AS obj ON                               -- original memberships child object id is joined to a t_object table, could be VARIABLE
        mem.child_object_id = obj.object_id
    INNER JOIN t_object AS tag_object ON                        -- tags can contain objects too, so joining another object table
        tag_object.object_id = tag.object_id
    LEFT JOIN t_action AS action ON
        tag.action_id = action.action_id
    LEFT JOIN t_data as nested_data ON                           -- here I am bringing in a another data table to join on the object id of the tag
        tag.object_id = nested_data.membership_id
    INNER JOIN t_text AS text ON
        text.data_id = nested_data.data_id
    
"""

schema = { 
    'collection_id': pl.Int32,
    'child_object_id': pl.Int32,
    'child_object_name': pl.String,
    'child_property_name': pl.String,
    'tag_data_id': pl.Int32,
    'tag_object_id': pl.Int32,
    'tag_object_name': pl.String,
    'tag_object_class_id': pl.Int32,
    'date_from': pl.String,
    'date_to': pl.String,
    'memo': pl.String,
    'action_symbol': pl.String,
    'tag_action_id': pl.Int32,
    'text': pl.String,
    'text_class_id': pl.Int32
}

In [113]:
qr = PlexosSQLite(xml_file).query(tag_cte)
# df = pl.from_records(qr,strict=False).to_pandas().T.to_csv('tag_cte.csv', index=False)
df = pl.from_records(qr, schema=schema)
df.write_csv('tag_cte.csv')

2024-08-06 14:28:16.452 | DEBUG    | plexosdb.sqlite:_create_table_schema:774 - Using /Users/kamrantehranchi/Local_Documents/plexosdb/src/plexosdb/schema.sql for creating plexos schema.
/var/folders/00/vqry7y9s78q3rvf_m_jmk_bc0000gn/T/ipykernel_56769/3311931311.py:3: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df = pl.from_records(qr, schema=schema)


In [ ]:
tag_cte = """
    SELECT
        obj.object_id AS parent_object_id,
        obj.name AS parent_object_name,
        prop.name AS parent_property_name,
        tag.data_id AS tag_data_id,
        tag.object_id AS tag_object_id,
        tag_object.name AS tag_object_name,
        tag_object.class_id AS tag_object_class_id,
        date_from.date AS date_from,
        date_to.date AS date_to,
        memo.value AS memo,
        action.action_symbol AS action_symbol,
        tag.action_id AS tag_action_id
    FROM
        t_membership AS mem
    LEFT JOIN t_data AS d ON
        mem.membership_id = d.membership_id
    LEFT JOIN t_property AS prop ON
        prop.property_id = d.property_id
    LEFT JOIN t_memo_data AS memo ON
        memo.data_id = d.data_id
    LEFT JOIN t_date_from AS date_from ON
        d.data_id = date_from.data_id
    LEFT JOIN t_date_to AS date_to ON
        d.data_id = date_to.data_id
    INNER JOIN t_tag AS tag ON
        tag.data_id = d.data_id
    INNER JOIN t_object AS obj ON
        mem.child_object_id = obj.object_id
    INNER JOIN t_object AS tag_object ON
        tag_object.object_id = tag.object_id
    LEFT JOIN t_action AS action ON
        tag.action_id = action.action_id
    LEFT JOIN t_data as nested_data ON
        tag.object_id = nested_data.membership_id
    
"""

schema = { 
    'parent_object_id': pl.Int32,
    'parent_object_name': pl.String,
    'parent_property_name': pl.String,
    'tag_data_id': pl.Int32,
    'tag_object_id': pl.Int32,
    'tag_object_name': pl.String,
    'tag_object_class_id': pl.Int32,
    'date_from': pl.String,
    'date_to': pl.String,
    'memo': pl.String,
    'action_symbol': pl.String,
    'tag_action_id': pl.Int32
}